# Transfer Learning approach - VGG16
This is the notebook model used to apply transfer learning to solve the classification task.

Import the needed libraries and connect Drive:

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import os
import tensorflow as tf
import numpy as np
import shutil

SEED = 1518
tf.random.set_seed(SEED)  

# Get current working directory
cwd = os.getcwd()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
os.getcwd()

In [ ]:
# Get dataset directory
dataset_dir = os.path.join(cwd, 'drive/My Drive/MaskDataset')
dataset_dir
train_dir =os.path.join(dataset_dir,'training')
test_dir = os.path.join(dataset_dir, 'test')
os.listdir(dataset_dir)

Split the training set into training and validation and create ImageDataGenerator objects.
Data augmentation is applied based on the True or False value for the if condition.
When applied, it consists in image rotation, shift, horizontal flip and zoom.
In both cases, the pixel values are normalized in order to have values between 0 and 1.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=5,
                                        height_shift_range=5,
                                        zoom_range=0.2,
                                        horizontal_flip=True,
                                        vertical_flip=False,
                                        fill_mode='nearest',
                                        rescale=1./255,
                                        validation_split=0.15)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255,
                                        validation_split=0.15)

Set batch size and image shape in which the elements of training and validation dataset will be created. Shuffle and seed are applied. Due to our previous separation of images into classes folders in the training set we can now set the class mode to categorical.

In [ ]:
# Batch size
bs = 12

# Output image shape
img_width = 256
img_height = 256

# Classes
num_classes = 3

# Generate Training & Validation 
train_gen = train_data_gen.flow_from_directory(train_dir,
                                               batch_size=bs,
                                               classes=None,
                                               class_mode='categorical', # automatic 1-hot encoding
                                               color_mode='rgb',        
                                               shuffle=True,
                                               subset='training',
                                               seed=SEED) 

valid_gen = train_data_gen.flow_from_directory(train_dir,
                                               batch_size=bs,
                                               classes=None,
                                               class_mode='categorical',
                                               color_mode='rgb',        
                                               shuffle=True,
                                               subset='validation',
                                               seed=SEED)

# Training set creation (85pc) and validation set creation(15pc)
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_height, img_width, 3], [None, num_classes]))

valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_height, img_width, 3], [None, num_classes]))

# Adding repetition of elements for the next epochs
train_dataset.repeat()
valid_dataset.repeat()

Load the model to use for transfer learning. In this case the chosen model is VGG16, directly available in the applications section of Keras. Loaded weights are the ones of the network trained on the *ImageNet* database. The fully-connected part is removed by setting include_top to False.

In [ ]:
# Load VGG16 Model

vgg = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

Create the model by setting which weights of the pretrained model have to be trained again on our training set and designing the new fully-connected part. Here only the last convolutional layer of the VGG16 is retrained.

In [ ]:
# Create Model

finetuning = True

if finetuning:
    freeze_until = 14   # layer from which we want to fine-tune
    
    for layer in vgg.layers[:freeze_until]:
        layer.trainable = False
else:
    vgg.trainable = False
    
model = tf.keras.Sequential()
model.add(vgg)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(units=64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

# Model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 8, 8, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               8388864   
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                16448     
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 3)                

Set optimization parameters: loss, optimizer and metrics

In [ ]:
# Loss function
loss = tf.keras.losses.CategoricalCrossentropy()

# Learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

# Validation metrics
metrics = ['accuracy']

In [ ]:
# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)


Load tensorboard (uncomment to load)

In [ ]:
#%reload_ext tensorboard
#%tensorboard --logdir /content/drive/My\ Drive/MaskDataset/classification_experiments --port 8008

Set callbacks and visualization of losses and metrics on Tensorboard

In [ ]:
# Callbacks
from datetime import datetime


cwd = os.getcwd()

exps_dir = os.path.join('/content/drive/My Drive/MaskDataset/', 'classification_experiments_VGG_3')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN_1'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Visualize Learning on Tensorboard
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# Show losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)
callbacks.append(tb_callback)

Activate Early Stopping to reduce overfitting

In [ ]:
# Early Stopping
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    callbacks.append(es_callback)

Fit the model after setting the number of epochs and the number of steps for each epoch.

In [ ]:
model.fit(x = train_dataset,
          epochs = 100,
          steps_per_epoch = len(train_gen),
          validation_data = valid_dataset,
          validation_steps = len(valid_gen), 
          callbacks = callbacks)


<FlatMapDataset shapes: ((None, 256, 256, 3), (None, 3)), types: (tf.float32, tf.float32)>

<FlatMapDataset shapes: ((None, 256, 256, 3), (None, 3)), types: (tf.float32, tf.float32)>

Epoch 1/100
399/399 [==============================] - 147s 369ms/step - loss: 0.8509 - accuracy: 0.5632 - val_loss: 0.6506 - val_accuracy: 0.6643
Epoch 2/100
399/399 [==============================] - 146s 366ms/step - loss: 0.6173 - accuracy: 0.6966 - val_loss: 0.5137 - val_accuracy: 0.7628
Epoch 3/100
399/399 [==============================] - 146s 366ms/step - loss: 0.5523 - accuracy: 0.7466 - val_loss: 0.5701 - val_accuracy: 0.7307
Epoch 4/100
399/399 [==============================] - 146s 365ms/step - loss: 0.4996 - accuracy: 0.7771 - val_loss: 0.4803 - val_accuracy: 0.7805
Epoch 5/100
399/399 [==============================] - 146s 366ms/step - loss: 0.4293 - accuracy: 0.8125 - val_loss: 0.4143 - val_accuracy: 0.8197
Epoch 6/100
399/399 [==============================] - 146s 367ms/step - loss: 0.3974 - accuracy: 0.8246 - val_loss: 0.4238 - val_accuracy: 0.8316
Epoch 7/100
399/399 [==============================] - 147s 369ms/step - loss: 0.3714 - accuracy: 0.8424 - val_loss: 0

Test the model on the test set and make predictions.

In [ ]:
import os
from PIL import Image
cwd = os.getcwd()

dir = os.path.join('/content/drive/My Drive/MaskDataset/', 'test')
results = {}
for subdir, dirs, files in os.walk(dir):
    for file in files:
        filepath = subdir + os.sep + file
        img = Image.open(filepath).convert('RGB')
        img = img.resize((img_height,img_width))
        img_array = np.array(img)
        img_array = np.expand_dims(img_array, 0) 
        img_tensor = tf.convert_to_tensor(img_array)
        img_tensor = tf.cast(img_array,tf.float32)/255.
        softmax = model.predict(img_tensor)
        prediction = np.argmax(softmax)   # predicted class
        results[file] = prediction
        
        


Create the CSV file from the obtained predictions that will be submitted in order to have a score for the model.

In [ ]:
import os
from datetime import datetime

def create_csv(results, results_dir='/content/drive/My Drive/Progetto/Results'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')